In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import shutil

from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics
!wget -O yolov8s.pt https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8s.pt

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-

# **Why Choose YOLO?**

Yolo is extremely fast (real-time detection) with respect to R-CNN Faster. It is also optimized for video processing (e.g., sports tracking, self-driving cars) and it is lighter and can run on edge devices (mobile, embedded systems).

Although it's accuracy is not as high as R-CNN Faster, I chose this model because the advantages outweigh, especially it's speed.





In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ayushspai/sportsmot/versions/1")

print("Path to dataset files:", path)

# Move data from it's directory to colab
!mv {path} /content/SportsMOT
print("Dataset moved to /content/SportsMOT")

In [ ]:
football_videos = []
with open("/content/SportsMOT/sportsmot_publish/splits_txt/football.txt", "r") as f:
    football_videos = [line.strip() for line in f.readlines()]

print("Football Video Files:", football_videos[:5])  # Show first 5 filenames

In [ ]:
import os
import shutil

dataset_root = "SportsMOT/sportsmot_publish/dataset"

train_dir = os.path.join(dataset_root, "train")
val_dir = os.path.join(dataset_root, "val")
test_dir = os.path.join(dataset_root, "test")

filtered_train_dir = os.path.join(dataset_root, "train_football")
filtered_val_dir = os.path.join(dataset_root, "val_football")
filtered_test_dir = os.path.join(dataset_root, "test_football")

for path in [filtered_train_dir, filtered_val_dir, filtered_test_dir]:
    os.makedirs(path, exist_ok=True)

def filter_and_copy_football_videos(original_dir, filtered_dir, football_videos):
    for folder in os.listdir(original_dir):
        if folder in football_videos:
            src_path = os.path.join(original_dir, folder)
            dest_path = os.path.join(filtered_dir, folder)
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
            print(f"Copied: {folder}")

filter_and_copy_football_videos(train_dir, filtered_train_dir, football_videos)
filter_and_copy_football_videos(val_dir, filtered_val_dir, football_videos)
filter_and_copy_football_videos(test_dir, filtered_test_dir, football_videos)

print("Football dataset filtering complete!")


In [ ]:
def convert_mot_to_yolo(video_list, dataset_split):
    dataset_path = f"SportsMOT/sportsmot_publish/dataset/{dataset_split}_football"
    yolo_labels_path = f"SportsMOT/sportsmot_publish/dataset/{dataset_split}_football/labels"

    os.makedirs(yolo_labels_path, exist_ok=True)

    for video in video_list:
        gt_path = os.path.join(dataset_path, video, "gt", "gt.txt")
        img_path = os.path.join(dataset_path, video, "img1")

        if not os.path.exists(gt_path):
            print(f"Skipping {video}: No gt.txt found!")
            continue

        gt_df = pd.read_csv(gt_path, header=None)
        gt_df.columns = ["frame_id", "object_id", "x", "y", "width", "height", "confidence", "class", "visibility"]

        for frame_id in gt_df["frame_id"].unique():
            frame_data = gt_df[gt_df["frame_id"] == frame_id]

            # YOLO label file per frame
            label_filename = os.path.join(yolo_labels_path, f"{frame_id:06d}.txt")
            with open(label_filename, "w") as f:
                for _, row in frame_data.iterrows():
                    x_center = (row["x"] + row["width"] / 2) / 1280  # Normalize x (width = 1280)
                    y_center = (row["y"] + row["height"] / 2) / 720   # Normalize y (height = 720)
                    width = row["width"] / 1280
                    height = row["height"] / 720
                    class_id = int(row["class"])  # Use class ID

                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        print(f"Converted: {video}")


convert_mot_to_yolo(os.listdir("SportsMOT/sportsmot_publish/dataset/train_football"), "train")
convert_mot_to_yolo(os.listdir("SportsMOT/sportsmot_publish/dataset/val_football"), "val")
convert_mot_to_yolo(os.listdir("SportsMOT/sportsmot_publish/dataset/test_football"), "test")

print("✅ MOT → YOLO label conversion complete!")


In [ ]:
# YOLO dataset paths
yolo_train_path = "SportsMOT/sportsmot_publish/dataset/yolo/train"
yolo_val_path = "SportsMOT/sportsmot_publish/dataset/yolo/val"
yolo_test_path = "SportsMOT/sportsmot_publish/dataset/yolo/test"

for path in [yolo_train_path, yolo_val_path, yolo_test_path]:
    os.makedirs(os.path.join(path, "images"), exist_ok=True)
    os.makedirs(os.path.join(path, "labels"), exist_ok=True)

def move_and_rename_files(video_list, dataset_split):
    dataset_path = f"SportsMOT/sportsmot_publish/dataset/{dataset_split}"
    yolo_path = f"SportsMOT/sportsmot_publish/dataset/yolo/{dataset_split.replace('_football', '')}"  # Convert train_football → train

    for video in video_list:
        img_src = os.path.join(dataset_path, video, "img1")
        label_src = os.path.join(dataset_path, "labels")

        img_dest = os.path.join(yolo_path, "images")
        label_dest = os.path.join(yolo_path, "labels")

        os.makedirs(img_dest, exist_ok=True)
        os.makedirs(label_dest, exist_ok=True)

        # Rename and copy images
        if os.path.exists(img_src):
            for img in os.listdir(img_src):
                new_img_name = f"{video}_{img}"
                shutil.copy(os.path.join(img_src, img), os.path.join(img_dest, new_img_name))
            print(f"Moved and renamed images for: {video}")
        else:
            print(f"Skipping images: {video} (img1/ folder not found)")

        # Rename and copy labels
        if os.path.exists(label_src):
            for lbl in os.listdir(label_src):
                new_lbl_name = f"{video}_{lbl}"  # Rename: v_1yHWGw8DH4A_c029_000001.txt
                shutil.copy(os.path.join(label_src, lbl), os.path.join(label_dest, new_lbl_name))
            print(f"Moved and renamed labels for: {video}")
        else:
            print(f"Skipping labels: {video} (labels/ folder not found)")

move_and_rename_files(os.listdir("SportsMOT/sportsmot_publish/dataset/train_football"), "train_football")
move_and_rename_files(os.listdir("SportsMOT/sportsmot_publish/dataset/val_football"), "val_football")
move_and_rename_files(os.listdir("SportsMOT/sportsmot_publish/dataset/test_football"), "test_football")

print("YOLO dataset organized correctly with unique filenames.")

In [ ]:
dataset_yaml = """
path: /content/SportsMOT/sportsmot_publish/dataset/yolo
train: /content/SportsMOT/sportsmot_publish/dataset/yolo/train/images
val: /content/SportsMOT/sportsmot_publish/dataset/yolo/val/images
test: /content/SportsMOT/sportsmot_publish/dataset/yolo/test/images

names:
  0: player
  1: ball
"""

yaml_path = "/content/SportsMOT/sportsmot_publish/dataset/yolo/dataset.yaml"
os.makedirs(os.path.dirname(yaml_path), exist_ok=True)

with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"Dataset configuration updated at {yaml_path}.")

# Train YOLO

### Load Data from Drive to Colab to Run

In [ ]:
# !cp -r /content/drive/MyDrive/yolo /content/

I first ran it for 15 epochs, then did it for 5 epochs. So it ran for 20 epochs.

In [ ]:
from ultralytics import YOLO
data_path = "/content/SportsMOT/sportsmot_publish/dataset/yolo/dataset.yaml"

model = YOLO("yolov8s.pt")

model.train(data=data_path, epochs=15, imgsz=720, device="cuda")

!tensorboard --logdir runs/detect/train


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the trained model to Google Drive
!cp runs/detect/train3/weights/best.pt /content/drive/MyDrive/yolo/best.pt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load training results
metrics = pd.read_csv("runs/detect/train/results.csv")

# Extract values
epochs = metrics["epoch"]
train_loss = metrics["train/box_loss"]
val_loss = metrics["val/box_loss"]
map50 = metrics["metrics/mAP_50"]
map50_95 = metrics["metrics/mAP_50-95"]

# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label="Train Loss")
plt.plot(epochs, val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.show()

# Plot accuracy curves
plt.figure(figsize=(10, 5))
plt.plot(epochs, map50, label="mAP@50")
plt.plot(epochs, map50_95, label="mAP@50-95")
plt.xlabel("Epochs")
plt.ylabel("mAP Score")
plt.title("Mean Average Precision (mAP) Over Epochs")
plt.legend()
plt.show()

In [ ]:
metrics = model.val(data=data_path)
print(f"mAP@50: {metrics['metrics/mAP_50']:.4f}")
print(f"mAP@50-95: {metrics['metrics/mAP_50-95']:.4f}")
print(f"Precision: {metrics['metrics/precision']:.4f}")
print(f"Recall: {metrics['metrics/recall']:.4f}")

In [ ]:
# Run inference on test images
model.predict(source="/content/SportsMOT/sportsmot_publish/yolo/test/images", save=True, conf=0.5)

import cv2
import matplotlib.pyplot as plt

# Load and display a sample image with predictions
img_path = "runs/detect/predict/image1.jpg"
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 5))
plt.imshow(image)
plt.axis("off")
plt.title("YOLOv8 Predictions on Sample Frame")
plt.show()


## SOT

In [ ]:
import os
import cv2
import numpy as np
import glob

# Path to the folder containing test images
test_folder = "/content/SportsMOT/sportsmot_publish/yolo/test/images"

# Get all image filenames
image_files = sorted(glob.glob(os.path.join(test_folder, "*.jpg")))

# Ensure correct ordering by sorting numerically
image_files.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))

print(f"Total frames found: {len(image_files)}")
print(f"First frame: {image_files[0]}")
print(f"Last frame: {image_files[-1]}")

In [ ]:
# Read the first frame
first_frame = cv2.imread(image_files[0])

# Manually select the object to track
bbox = cv2.selectROI("Select Object to Track", first_frame, fromCenter=False, showCrosshair=True)

# Initialize CSRT Tracker
tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, bbox)

cv2.destroyAllWindows()

In [ ]:
# List to store object trajectory
trajectory = []

for img_path in image_files:
    frame = cv2.imread(img_path)

    # Update tracker
    success, bbox = tracker.update(frame)

    if success:
        x, y, w, h = map(int, bbox)
        center = (x + w // 2, y + h // 2)
        trajectory.append(center)

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, center, 3, (0, 0, 255), -1)

    # Draw trajectory path
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i - 1], trajectory[i], (255, 0, 0), 2)

    # Display frame
    cv2.imshow("Object Tracking", frame)
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()

In [ ]:
output_video_path = "/content/tracking_output.avi"
frame_size = cv2.imread(image_files[0]).shape[1::-1]  # Get frame dimensions (width, height)

# Define video writer
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"XVID"), 30, frame_size)

for img_path in image_files:
    frame = cv2.imread(img_path)
    success, bbox = tracker.update(frame)

    if success:
        x, y, w, h = map(int, bbox)
        center = (x + w // 2, y + h // 2)
        trajectory.append(center)

        # Draw bounding box and trajectory
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, center, 3, (0, 0, 255), -1)
        for i in range(1, len(trajectory)):
            cv2.line(frame, trajectory[i - 1], trajectory[i], (255, 0, 0), 2)

    out.write(frame)

out.release()
print(f"Tracking video saved at {output_video_path}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create an empty heatmap
heatmap = np.zeros((720, 1280))

# Increase heatmap intensity at tracked positions
for (x, y) in trajectory:
    heatmap[y, x] += 1

# Display heatmap
plt.figure(figsize=(10, 5))
sns.heatmap(heatmap, cmap="hot", cbar=True)
plt.title("Object Movement Heatmap")
plt.show()
